In [1]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPool2D
import keras
import keras.backend as k
from keras.models import Sequential
from sklearn.preprocessing import LabelBinarizer
import tensorflow as tf      
import logging
logging.getLogger('tensorflow').setLevel(logging.ERROR)
k.clear_session()
from keras.regularizers import l2
from keras.layers import BatchNormalization

In [2]:



train_datagen =ImageDataGenerator(width_shift_range=0.15,
        height_shift_range=0.15, shear_range=0.15,
       
        validation_split=0.15,
        rescale=1./255)
test_datagen =ImageDataGenerator(width_shift_range=0.15,
        height_shift_range=0.15, shear_range=0.15,
  
        validation_split=0.15,
        rescale=1./255)
train_data = train_datagen.flow_from_directory(
       r'/content/drive/MyDrive/dataset-corrupted/n+c+g/train',
        target_size=(256,256),
        batch_size=210,subset ='training',
        class_mode='categorical')
valid_data = train_datagen.flow_from_directory(
       r'/content/drive/MyDrive/dataset-corrupted/n+c+g/train',
        target_size=(256,256),
        batch_size=128,subset ='validation',
        class_mode='categorical')
test_data = test_datagen.flow_from_directory(
       r'/content/drive/MyDrive/dataset-corrupted/n+c+g/test',
        target_size=(256,256),
        batch_size=128,
        class_mode='categorical')

Found 9804 images belonging to 50 classes.
Found 1700 images belonging to 50 classes.
Found 3650 images belonging to 50 classes.


In [3]:
def combine_gen(*gens):
    while True:
        for g in gens:
            yield next(g)


In [2]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(filters = 8, kernel_size = (11,11), kernel_regularizer=l2(0.001), padding = 'Same', activation ='relu', input_shape = (256,256,3)))
model.add(tf.keras.layers.Conv2D(filters = 48, kernel_size = (5,5), kernel_regularizer=l2(0.001), padding = 'Same', activation ='relu'))
model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2), strides = 2))
model.add(tf.keras.layers.Conv2D(filters = 128, kernel_size = (3,3),kernel_regularizer=l2(0.001), padding = 'Same', activation ='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2), strides= 2))
model.add(tf.keras.layers.Conv2D(filters = 192, kernel_size = (3,3), kernel_regularizer=l2(0.001), padding = 'Same', activation ='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2), strides= 2))
model.add(tf.keras.layers.Conv2D(filters = 192, kernel_size = (3,3), kernel_regularizer=l2(0.001), padding = 'Same', activation ='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2), strides= 2))

model.add(tf.keras.layers.Flatten())

model.add(tf.keras.layers.Dense(512, activation = "relu", kernel_regularizer=l2(0.001)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dense(256, activation = "relu", kernel_regularizer=l2(0.001)))
model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Dense(50, activation = "softmax"))
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 256, 256, 8)       2912      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 256, 256, 48)      9648      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 128, 128, 48)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 128, 128, 128)     55424     
_________________________________________________________________
batch_normalization (BatchNo (None, 128, 128, 128)     512       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 64, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 64, 64, 192)       2

In [5]:
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.callbacks import CSVLogger

csv = CSVLogger("/content/drive/MyDrive/normal+corrupted/augumentated-corrupte+normal-cnn-50-model_history_log(normal).csv", append=True)
es1= tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',patience=5, restore_best_weights=True)
mc = ModelCheckpoint('/content/drive/MyDrive/normal+corrupted/augumentated-corrupte+normal-cnn-50-model.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

In [3]:
adam = keras.optimizers.Adam(learning_rate = 0.01, decay = 0.9)
model.compile(optimizer = adam, loss = "categorical_crossentropy", metrics = ["accuracy"])

In [7]:
history =model.fit(train_data, validation_data= valid_data,epochs = 300,callbacks = [mc,csv,es1],shuffle =True )

Epoch 1/300
47/47 [==============================] - 3964s 84s/step - loss: 12.3141 - accuracy: 0.2403 - val_loss: 134.6772 - val_accuracy: 0.0235

Epoch 00001: val_accuracy improved from -inf to 0.02353, saving model to /content/drive/MyDrive/normal+corrupted/augumentated-corrupte+normal-cnn-50-model.h5
Epoch 2/300
47/47 [==============================] - 442s 9s/step - loss: 10.6097 - accuracy: 0.5882 - val_loss: 42.7417 - val_accuracy: 0.0359

Epoch 00002: val_accuracy improved from 0.02353 to 0.03588, saving model to /content/drive/MyDrive/normal+corrupted/augumentated-corrupte+normal-cnn-50-model.h5
Epoch 3/300
47/47 [==============================] - 432s 9s/step - loss: 9.4039 - accuracy: 0.7398 - val_loss: 20.9974 - val_accuracy: 0.0241

Epoch 00003: val_accuracy did not improve from 0.03588
Epoch 4/300
47/47 [==============================] - 423s 9s/step - loss: 8.6149 - accuracy: 0.8324 - val_loss: 12.2233 - val_accuracy: 0.0482

Epoch 00004: val_accuracy improved from 0.035

KeyboardInterrupt: ignored

In [22]:
model.save_weights('/content/drive/MyDrive/normal+corrupted/augumentated-corrupte+normal-cnn-dataset-50_weights.h5')

In [5]:
model.load_weights('augumentated-corrupte+normal-cnn-dataset-50_weights.h5')

In [8]:

train_acc = model.evaluate(train_data,steps=80)
val_acc = model.evaluate(valid_data,steps=80)
print("train_Accuracy: %.2f%%" % (train_acc[1]*100))
print("valid_Accuracy: %.2f%%" % (val_acc[1]*100))

80/80 [==============================] - 62s 735ms/step - loss: 6.3401 - accuracy: 0.9276
train_Accuracy: 95.93%
valid_Accuracy: 92.76%


In [6]:
test_datagen_nor = ImageDataGenerator(rescale=1./255)

test_datagen_aug =  ImageDataGenerator(width_shift_range=0.15,
        height_shift_range=0.15, shear_range=0.15,
       
        rescale=1./255)

In [ ]:
## normal dataset

In [12]:
test_genrator_nor = test_datagen_nor.flow_from_directory( r'/content/drive/MyDrive/dataset-50/nor-dataset-50/test',
       batch_size=128,target_size=(256,256),
        class_mode='categorical')
test_acc = model.evaluate(test_genrator_nor)
print("Test_Accuracy: %.2f%%" % (test_acc[1]*100))

Found 1300 images belonging to 50 classes.
11/11 [==============================] - 17s 1s/step - loss: 6.1020 - accuracy: 0.9585
Test_Accuracy: 95.85%


In [ ]:
## normal dataset +augumenattion

In [10]:
test_genrator_nor = test_datagen_aug.flow_from_directory( r'/content/drive/MyDrive/dataset-50/nor-dataset-50/test',
       batch_size=128,target_size=(256,256),
        class_mode='categorical')
test_acc = model.evaluate(test_genrator_nor)
print("Test_Accuracy: %.2f%%" % (test_acc[1]*100))

Found 1300 images belonging to 50 classes.
11/11 [==============================] - 410s 41s/step - loss: 6.0639 - accuracy: 0.9662
Test_Accuracy: 96.62%


In [ ]:
## gausian noise

In [13]:
test_genrator_gau = test_datagen_nor.flow_from_directory( r'/content/drive/MyDrive/dataset-50/gaussian-50/test',
       batch_size=128,target_size=(256,256),
        class_mode='categorical')
test_acc = model.evaluate(test_genrator_gau)
print("Test_Accuracy: %.2f%%" % (test_acc[1]*100))

Found 1300 images belonging to 50 classes.
11/11 [==============================] - 553s 55s/step - loss: 6.3217 - accuracy: 0.9131
Test_Accuracy: 91.31%


In [ ]:
## gaussian augumentation

In [18]:
test_genrator_gau = test_datagen_aug.flow_from_directory( r'/content/drive/MyDrive/dataset-50/gaussian-50/test',
       batch_size=128,target_size=(256,256),
        class_mode='categorical')
test_acc = model.evaluate(test_genrator_gau)
print("Test_Accuracy: %.2f%%" % (test_acc[1]*100))

Found 1300 images belonging to 50 classes.
11/11 [==============================] - 71s 6s/step - loss: 6.1699 - accuracy: 0.9623
Test_Accuracy: 96.23%


In [ ]:
## corrupted 10

In [7]:
test_genrator_cor10 = test_datagen_nor.flow_from_directory( r'D:\Uni-passau\thesis-smalldataset\corrupted-10',
       batch_size=128,target_size=(256,256),
        class_mode='categorical')
test_acc = model.evaluate(test_genrator_cor10)
print("Test_Accuracy: %.2f%%" % (test_acc[1]*100))

Found 1300 images belonging to 50 classes.
11/11 [==============================] - 70s 6s/step - loss: 6.7298 - accuracy: 0.8223
Test_Accuracy: 82.23%


In [ ]:
## corrupted 10 augu

In [16]:
test_genrator_cor10_aug = test_datagen_aug.flow_from_directory( r'D:\Uni-passau\thesis-smalldataset\corrupted-10',
       batch_size=128,target_size=(256,256),
        class_mode='categorical')
test_acc = model.evaluate(test_genrator_cor10_aug)
print("Test_Accuracy: %.2f%%" % (test_acc[1]*100))

Found 1300 images belonging to 50 classes.
11/11 [==============================] - 88s 8s/step - loss: 6.4207 - accuracy: 0.9115
Test_Accuracy: 91.15%


In [ ]:
## corrupted 20

In [15]:
test_genrator_cor20 = test_datagen_nor.flow_from_directory( r'/content/drive/MyDrive/dataset-50/corrupted-1/test',
       batch_size=128,target_size=(256,256),
        class_mode='categorical')
test_acc = model.evaluate(test_genrator_cor20)
print("Test_Accuracy: %.2f%%" % (test_acc[1]*100))

Found 1300 images belonging to 50 classes.
11/11 [==============================] - 350s 35s/step - loss: 6.6157 - accuracy: 0.8692
Test_Accuracy: 86.92%


In [ ]:
## corrupted 20 augumnetation

In [19]:
test_genrator_cor20 = test_datagen_aug.flow_from_directory( r'/content/drive/MyDrive/dataset-50/corrupted-1/test',
       batch_size=128,target_size=(256,256),
        class_mode='categorical')
test_acc = model.evaluate(test_genrator_cor20)
print("Test_Accuracy: %.2f%%" % (test_acc[1]*100))

Found 1300 images belonging to 50 classes.
11/11 [==============================] - 32s 3s/step - loss: 6.4416 - accuracy: 0.9277
Test_Accuracy: 92.77%


In [ ]:
## corrupted -30

In [9]:
test_genrator_cor30 = test_datagen_nor.flow_from_directory( r'D:\Uni-passau\thesis-smalldataset\corrupted-30',
       batch_size=128,target_size=(256,256),
        class_mode='categorical')
test_acc = model.evaluate(test_genrator_cor30)
print("Test_Accuracy: %.2f%%" % (test_acc[1]*100))

Found 1300 images belonging to 50 classes.
11/11 [==============================] - 68s 6s/step - loss: 6.7711 - accuracy: 0.8454
Test_Accuracy: 84.54%


In [12]:
test_genrator_cor30_aug = test_datagen_aug.flow_from_directory( r'D:\Uni-passau\thesis-smalldataset\corrupted-30',
       batch_size=128,target_size=(256,256),
        class_mode='categorical')
test_acc = model.evaluate(test_genrator_cor30_aug)
print("Test_Accuracy: %.2f%%" % (test_acc[1]*100))

Found 1300 images belonging to 50 classes.
11/11 [==============================] - 86s 8s/step - loss: 6.6520 - accuracy: 0.9023
Test_Accuracy: 90.23%


In [ ]:
## corrupted- 40

In [13]:
test_genrator_cor40 = test_datagen_nor.flow_from_directory( r'D:\Uni-passau\thesis-smalldataset\corrupted-40',
       batch_size=128,target_size=(256,256),
        class_mode='categorical')
test_acc = model.evaluate(test_genrator_cor40)
print("Test_Accuracy: %.2f%%" % (test_acc[1]*100))

Found 1300 images belonging to 50 classes.
11/11 [==============================] - 71s 6s/step - loss: 7.4614 - accuracy: 0.5962
Test_Accuracy: 59.62%


In [14]:
## corrupted - 40 augu

In [15]:
test_genrator_cor40_aug = test_datagen_aug.flow_from_directory( r'D:\Uni-passau\thesis-smalldataset\corrupted-40',
       batch_size=128,target_size=(256,256),
        class_mode='categorical')
test_acc = model.evaluate(test_genrator_cor40_aug)
print("Test_Accuracy: %.2f%%" % (test_acc[1]*100))

Found 1300 images belonging to 50 classes.
11/11 [==============================] - 85s 8s/step - loss: 7.9126 - accuracy: 0.5323
Test_Accuracy: 53.23%


In [ ]:
## corrupted-25

In [14]:
test_corrupted= test_datagen_nor.flow_from_directory( r'/content/drive/MyDrive/dataset-corrupted/corrupted-25/test',
        batch_size=32,target_size=(256,256),
        class_mode='categorical')
test_acc = model.evaluate(test_corrupted)
print("Test_Accuracy: %.2f%%" % (test_acc[1]*100))

Found 1050 images belonging to 50 classes.
33/33 [==============================] - 277s 9s/step - loss: 6.5424 - accuracy: 0.9076
Test_Accuracy: 90.76%


In [ ]:
## corrupted -25 augu

In [17]:
test_corrupted= test_datagen_aug.flow_from_directory( r'/content/drive/MyDrive/dataset-corrupted/corrupted-25/test',
        batch_size=32,target_size=(256,256),
        class_mode='categorical')
test_acc = model.evaluate(test_corrupted)
print("Test_Accuracy: %.2f%%" % (test_acc[1]*100))

Found 1050 images belonging to 50 classes.
33/33 [==============================] - 26s 783ms/step - loss: 6.4378 - accuracy: 0.9429
Test_Accuracy: 94.29%


In [20]:
test_genrator_cor50 = test_datagen_nor.flow_from_directory( r'/content/drive/MyDrive/dataset-50/corrupted-2/test',
       batch_size=128,target_size=(256,256),
        class_mode='categorical')
test_acc = model.evaluate(test_genrator_cor50)
print("Test_Accuracy: %.2f%%" % (test_acc[1]*100))

Found 1300 images belonging to 50 classes.
11/11 [==============================] - 150s 15s/step - loss: 9.9771 - accuracy: 0.0954
Test_Accuracy: 9.54%


In [21]:
test_genrator_cor50 = test_datagen_aug.flow_from_directory( r'/content/drive/MyDrive/dataset-50/corrupted-2/test',
       batch_size=128,target_size=(256,256),
        class_mode='categorical')
test_acc = model.evaluate(test_genrator_cor50)
print("Test_Accuracy: %.2f%%" % (test_acc[1]*100))

Found 1300 images belonging to 50 classes.
11/11 [==============================] - 30s 3s/step - loss: 10.5437 - accuracy: 0.2431
Test_Accuracy: 24.31%


In [ ]:
test_genrator_cor60_aug = test_datagen_aug.flow_from_directory( r'D:\Uni-passau\thesis-smalldataset\corrupted-60',
       batch_size=128,target_size=(256,256),
        class_mode='categorical')
test_acc = model.evaluate(test_genrator_cor60_aug)
print("Test_Accuracy: %.2f%%" % (test_acc[1]*100))

Found 1300 images belonging to 50 classes.
 9/11 [=======================>......] - ETA: 19s - loss: 16.9054 - accuracy: 0.0668

In [ ]:
test_genrator_cor60 = test_datagen_nor.flow_from_directory( r'D:\Uni-passau\thesis-smalldataset\corrupted-60',
       batch_size=128,target_size=(256,256),
        class_mode='categorical')
test_acc = model.evaluate(test_genrator_cor60)
print("Test_Accuracy: %.2f%%" % (test_acc[1]*100))